In [1]:
import numpy as np
import pandas as pd
import csv as csv
from sklearn.ensemble import RandomForestClassifier

In [2]:
# reads in train and test data sets

train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

In [3]:
train_set['Distance_To_Hydrology'] = np.sqrt(train_set.Vertical_Distance_To_Hydrology**2 + 
                                             train_set.Horizontal_Distance_To_Hydrology**2)
test_set['Distance_To_Hydrology'] = np.sqrt(test_set.Vertical_Distance_To_Hydrology**2 + 
                                            test_set.Horizontal_Distance_To_Hydrology**2)

In [4]:
ids = test_set['Id'].values
train_data = train_set.drop(['Id', 'Cover_Type'], axis = 1)
test_data = test_set.drop(['Id'], axis = 1)
target_data = train_set.Cover_Type.values

In [5]:
train_values = train_data.values
test_values = test_data.values

In [7]:
rf = RandomForestClassifier(n_estimators=500)
rf = rf.fit(train_values, target_data)

In [8]:
output = rf.predict(test_values).astype(int)

In [9]:
predictions_file = open("forest_cover_prediction.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(['Id', 'Cover_Type'])
open_file_object.writerows(zip(ids, output))
predictions_file.close()
print('Done.')

Done.


In [10]:
prediction = pd.read_csv('forest_cover_prediction.csv')
prediction.Cover_Type.describe()

count    565892.000000
mean          2.337547
std           1.732320
min           1.000000
25%           1.000000
50%           2.000000
75%           2.000000
max           7.000000
Name: Cover_Type, dtype: float64

In [11]:
pd.DataFrame(rf.feature_importances_, train_data.columns).sort([0], ascending = False) [:10]

,0
Elevation,0.216579
Horizontal_Distance_To_Roadways,0.086956
Horizontal_Distance_To_Fire_Points,0.070301
Distance_To_Hydrology,0.053384
Hillshade_9am,0.048797
Aspect,0.047833
Horizontal_Distance_To_Hydrology,0.047492
Wilderness_Area4,0.045709
Vertical_Distance_To_Hydrology,0.044778
Hillshade_3pm,0.043358
